## Release and Tag mining

- Get and store all tags and releases from selected repositories for rq1.

In [9]:
repos = getProjects()
i=0
for repo in repos:
    project = repo[0]
    i += 1
    print('\n\n\n{}/{} Project - {}\n\n\n'.format(i,len(repos),project))
    
    mineTags(project)
    mineReleases(project)
    




1/4 Project - FTXInfotech/laravel-adminpanel





Error while processing storeTag >>> Exception: duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy9iZXRh) already exists.

duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy9iZXRh) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/git/commits/b0204f643ffd43a7c50667aa51c369fa2baf6e8b




Error while processing storeTag >>> Exception: duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy92MS4wLWJldGE=) already exists.

duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy92MS4wLWJldGE=) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adm



Error while processing storeTag >>> Exception: duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy81LjguNA==) already exists.

duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy81LjguNA==) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/git/commits/9247e06b20c0ce48873ccfc8afab8a5b8db763f7




Error while processing storeTag >>> Exception: duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy81LjguNQ==) already exists.

duplicate key value violates unique constraint "tag_pkey"
DETAIL:  Key (node_id)=(MDM6UmVmMTA1Nzg5ODI0OnJlZnMvdGFncy81LjguNQ==) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/git/commits/19e50902deff2f603b2bde2eee266f9



Error while processing storeRelease >>> Exception: duplicate key value violates unique constraint "releases_pkey"
DETAIL:  Key (node_id)=(MDc6UmVsZWFzZTEzNjAzNDk5) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/releases/13603499




Error while processing storeRelease >>> Exception: duplicate key value violates unique constraint "releases_pkey"
DETAIL:  Key (node_id)=(MDc6UmVsZWFzZTExODM0MTY1) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/releases/11834165




Error while processing storeRelease >>> Exception: duplicate key value violates unique constraint "releases_pkey"
DETAIL:  Key (node_id)=(MDc6UmVsZWFzZTExMzI2Mjcy) already exists.

Project: FTXInfotech/laravel-adminpanel    Commit - https://api.github.com/repos/FTXInfotech/laravel-adminpanel/releases/11326272




Error while processing storeRelease >>> Exception: du

In [1]:
import psycopg2
import os
import requests 
import time
from datetime import datetime, timedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def connectDB():
    f = open('/home/elieziosoares/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "postgres",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v3")

In [3]:
def mineTags(project):
    url = 'https://api.github.com/repos/{}/git/refs/tags'.format(project)
    tags = requestAPI(url)
    
    if tags is None:
        return
    
    for tag in tags:
        try:
            ref = tag['ref']
            node = tag['node_id']
            url = tag['url']
            sha = tag['object']['sha']
            obj_type = tag['object']['type']
            obj_url = tag['object']['url']
            date = requestAPI(obj_url)
            
            if obj_type == 'tag':
                date = date['tagger']['date']
            else:
                date = date['committer']['date']
            
            storeTag(project,node,ref,url,sha,obj_type,obj_url,date)
            print('\t\t {} - {}'.format(date,ref))
        
        except Exception as e:
            print('\n\n==============================================================')
            print('Error while processing storeTag >>> Exception: {}'.format(e))
            print(e)
            print('Project: {}    Commit - {}\n\n'.format(project, obj_url))
        


In [4]:
def mineReleases(project):
    url = 'https://api.github.com/repos/{}/releases'.format(project)
    releases = requestAPI(url)
    
    if releases is None:
        return
    
    for release in releases:
        try:
            ref = release['tag_name']
            node = release['node_id']
            url = release['url']
            body = release['body']
            date = release['created_at']
            
            storeRelease(project,node,ref,url,date,body)
            print('\t\t {} - {}'.format(date,ref))
        
        except Exception as e:
            print('\n\n==============================================================')
            print('Error while processing storeRelease >>> Exception: {}'.format(e)) 
            print('Project: {}    Commit - {}\n\n'.format(project, url))
        


In [5]:
tokens =['token 80665cc2fa2fcf9f6368ac32014eda5bf5083bd3', 'token 1f3a6e7d066f937c163900db140e4506a2f3dd87', 'token 1f5163feb5b40cf70542f3a4360b957d9d6a668a']
i_token =0


def requestAPI(URL):
    # api-endpoint 
    global i_token
    
    if (len(tokens)-1) > i_token:
        i_token += 1
    else:
        i_token = 0
        
    r = requests.get(url = URL, headers={'Authorization': tokens[i_token],'Accept':'application/vnd.github.v3+json'}).json()
    #print(r)
    if len(r) > 0:
        try:
            #print('keys: {}'.format(r.keys()))
            
            if isinstance(r, dict) and 'message' in r.keys():
                #print('{} -- {}'.format(datetime.now().strftime("%H:%M:%S"),r['message']))
                if 'API rate limit exceeded' in r['message']:
                    time.sleep(600)
                    requestAPI(URL)
                elif 'Not Found' in r['message']:
                    return None
                else:
                    return r

            return r
        except Exception as e:
            print('\n Erro no request get: {}'.format(e))
            print(r)
    else:
        #print('-----')
        #print(r)
        return None

In [6]:
def getProjects():
    #query = """select repo_name from projects where ci_service is null AND 
    #    REPO_NAME IN (SELECT DISTINCT REPO_NAME FROM ISSUE WHERE ISBUG IS TRUE)
    #    AND REPO_NAME NOT IN (SELECT DISTINCT REPO_NAME FROM TAG)
    #    ORDER BY repo_name;"""
    
    query = """select repo_name from projects WHERE rq1_included is true and 
    repo_name in('AutoSpotting/AutoSpotting','docker-archive/deploykit','gitscrum-ce/laravel-gitscrum','mdbootstrap/mdb-ui-kit','openmediavault/openmediavault','printfcoder/stack-rpc-tutorials','SimpleMobileTools/Simple-Gallery')
    ORDER BY repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [7]:
def storeTag(repo_name,node_id,tag_ref,url,sha,object_type,object_url,object_date):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """INSERT INTO tag(
    repo_name,node_id,tag_ref,url,sha,object_type,object_url,object_date)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
    
    cursor.execute(query, [repo_name,node_id,tag_ref,url,sha,object_type,object_url,object_date])
    connection.commit()
    
    cursor.close()
    connection.close()

In [8]:
def storeRelease(repo_name,node_id,tag_ref,url,created_at,body):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """INSERT INTO releases(repo_name,node_id,tag_ref,url,created_at,body)
    VALUES (%s,%s,%s,%s,%s,%s)"""
    
    cursor.execute(query, [repo_name,node_id,tag_ref,url,created_at,body])
    connection.commit()
    
    cursor.close()
    connection.close()